<h1>2. fáza - predspracovanie dát</h1>

In [1254]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

import re

In [1255]:
def splitPersonalInfo(dataO):
    temp = dataO['personal_info'].str.split("\r\r\n|--|\\|", expand = True)
    dataO['ethn'] = temp[0]
    dataO['state'] = temp[1]
    dataO['edu'] = temp[2]
    dataO['status'] = temp[3]
    dataO['fam_memb'] = temp[4] 
    dataO.drop(['personal_info'], axis = 1, inplace = True)

In [1256]:
def fusionDuplicates(datasetO):
    duplicat = datasetO[datasetO['name'].duplicated(keep=False)]
    duplicat = duplicat.sort_values('name')
    duplicat = duplicat.reset_index(drop=True)
    
    for i, row in duplicat.iterrows():
        if(i%2==1):
            duplicat = duplicat.drop(i)
            continue
        for j, value in row.items():
            dupValue = duplicat.loc[i+1,j]
            if(pd.isna(value)):
                duplicat.loc[i,j]=dupValue
    datasetO.drop_duplicates(subset='name',keep=False,inplace=True)
    result = datasetO.append(duplicat, ignore_index=True)
    return(result)

In [1257]:
def correctValuesO(dataset):
    for col in dataset.columns:
        if(dataset[col].dtype != np.object):
           continue
        dataset[col] = dataset[col].str.replace('_','-')
        dataset[col] = dataset[col].replace(to_replace = r'.*\?.*', value = np.nan, regex = True)
        dataset[col] = dataset[col].replace(to_replace = r'[Nn][Aa][Nn]', value = np.nan, regex = True)
        dataset[col] = dataset[col].replace(['T','F'], ['t', 'f'])
        dataset[col] = dataset[col].replace(to_replace = r'[Tt][Rr][Uu][Ee]', value = 't', regex = True)
        dataset[col] = dataset[col].replace(to_replace = r'[Ff][Aa][Ll][Ss][Ee]', value = 'f', regex = True)

def correctDates(series):
    s = series.str.split("-", expand = True)
    x = 0
    while(x<len(series)):
        print(str(x)+" "+s.iloc[x])
        if(re.match(r'^[0-9][0-9]-[0-9][0-9]-[0-9][0-9]$', series.loc[x])):
            if(s.iloc[x][0][:1]=='0'):
                series.loc[x] = "20"+s.iloc[x][0]+"-"+s.iloc[x][1]+"-"+s.iloc[x][2][0:2]
            else:
                series.loc[x] = "19"+s.iloc[x][0]+"-"+s.iloc[x][1]+"-"+s.iloc[x][2][0:2]
#            x += 1
#            continue
         #if(len(s.iloc[x][2])>2):
         #   series.loc[x] = s.iloc[x][0]+"-"+s.iloc[x][1]+"-"+s.iloc[x][2][:2]
        if(re.match(r'^[0-9][0-9]-[0-9][0-9]-[0-9][0-9][0-9][0-9]$', series.loc[x])):
            y = s.iloc[x][0]
            s.iloc[x][0] = s.iloc[x][2]
            s.iloc[x][2] = y
            series.loc[x] = s.iloc[x][0]+"-"+s.iloc[x][1]+"-"+s.iloc[x][2][0:2]
        if(len(series.loc[x])>10):
            series.loc[x] = series.loc[x][0:10]
        x += 1
        
def correctValuesP(dataset):
    for col in dataset.columns:
        if(dataset[col].dtype != np.object):
           continue
        dataset[col] = dataset[col].str.replace('/','-')    
    correctDates(dataset["date_of_birth"])
        

In [1258]:
dataP = pd.read_csv("personal_train.csv", index_col=0)
dataO = pd.read_csv("other_train.csv", index_col=0)

In [1259]:
splitPersonalInfo(dataO)

In [1260]:
dataP.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3933 entries, 0 to 3932
Data columns (total 5 columns):
name             3933 non-null object
address          3933 non-null object
age              3933 non-null int64
sex              3933 non-null object
date_of_birth    3933 non-null object
dtypes: int64(1), object(4)
memory usage: 184.4+ KB


In [1261]:
dataO.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3983 entries, 0 to 3982
Data columns (total 25 columns):
name                3983 non-null object
address             3983 non-null object
kurtosis_oxygen     3709 non-null object
occupation          3974 non-null object
pregnant            3967 non-null object
education-num       3964 non-null float64
skewness_glucose    3970 non-null float64
mean_glucose        3573 non-null float64
std_oxygen          3967 non-null float64
capital-gain        3970 non-null float64
skewness_oxygen     3969 non-null float64
kurtosis_glucose    3965 non-null float64
fnlwgt              3974 non-null float64
class               3972 non-null float64
std_glucose         3964 non-null float64
income              3967 non-null object
mean_oxygen         3971 non-null float64
hours-per-week      3969 non-null float64
capital-loss        3971 non-null float64
workclass           3961 non-null object
ethn                3968 non-null object
state              

In [1262]:
len(dataO.name.unique())

3933

In [1263]:
namesO = dataO["name"].unique().sort()

In [1264]:
namesP = dataP["name"].unique().sort()

In [1265]:
np.array_equal(namesO, namesP)

True

In [1266]:
dataO.head()

,name,address,kurtosis_oxygen,occupation,pregnant,education-num,skewness_glucose,mean_glucose,std_oxygen,capital-gain,...,income,mean_oxygen,hours-per-week,capital-loss,workclass,ethn,state,edu,status,fam_memb
0,Daniel Love,"126 Ramirez Roads\r\nNew Jayville, IL 18861",10.58036423,Sales,f,9.0,3.158830,85.859375,14.121127,0.0,...,>50K,1.857860,40.0,1902.0,Private,White,United-States,HS-grad,Married-civ-spouse,Husband
1,Roy Paradis,"01168 Stark Pine Apt. 583\r\nDavidville, IN 32044",3.713364594,Craft-repair,f,9.0,1.234184,91.679688,36.946291,0.0,...,<=50K,10.001672,40.0,2002.0,Self-emp-not-inc,White,United-States,HS-grad,Married-civ-spouse,Husband
2,Charles Chen,"09482 Horne Lakes Apt. 373\r\nEast Brian, TN 2...",6.478064004,Handlers-cleaners,f,9.0,1.872355,89.960938,22.646980,0.0,...,<=50K,4.134615,48.0,0.0,Private,White,United-States,HS-grad,Married-civ-spouse,Husband
3,Chester Holmes,"720 Harris Turnpike Suite 732\r\nEast Sierra, ...",??,Sales,f,10.0,0.395583,NaN,14.001759,0.0,...,>50K,1.849498,40.0,0.0,Private,White,United-States,Some-college,Married-civ-spouse,Husband
4,James Ayala,Unit 3230 Box 9523\r\nDPO AA 55810,NaN,Transport-moving,f,9.0,0.493384,94.593750,29.361953,0.0,...,<=50K,5.739130,40.0,0.0,Private,Amer-Indian-Eskimo,United-States,HS-grad,Married-civ-spouse,Husband


In [1267]:
    dataOtypes = dataO["kurtosis_oxygen"].str.isdecimal()

In [1268]:
dataOtypes.shape

(3983,)

In [1269]:
dataOtypes.head()

0    False
1    False
2    False
3    False
4      NaN
Name: kurtosis_oxygen, dtype: object

In [1270]:
duplNames = dataO[dataO['name'].duplicated(keep=False)]
duplNames.sort_values('name')

,name,address,kurtosis_oxygen,occupation,pregnant,education-num,skewness_glucose,mean_glucose,std_oxygen,capital-gain,...,income,mean_oxygen,hours-per-week,capital-loss,workclass,ethn,state,edu,status,fam_memb
3718,Andrew Lawson,57159 Christopher Throughway Apt. 388\r\nWest ...,10.56049223,NaN,f,NaN,2.486716,87.585938,16.756233,0.0,...,>50K,2.035117,40.0,0.0,NaN,White,United-States,Bachelors,Married-civ-spouse,Husband
1298,Andrew Lawson,57159 Christopher Throughway Apt. 388\r\nWest ...,10.56049223,Sales,f,13.0,2.486716,87.585938,16.756233,NaN,...,NaN,2.035117,40.0,0.0,Private,White,United-States,Bachelors,Married-civ-spouse,Husband
3402,Andrew Olmstead,"519 Christopher Loop Suite 282\r\nRachelside, ...",6.9289193560000015,Craft-repair,f,9.0,0.548669,NaN,26.559749,0.0,...,<=50K,4.382943,52.0,0.0,NaN,White,United-States,HS-grad,Married-civ-spouse,Husband
281,Andrew Olmstead,"519 Christopher Loop Suite 282\r\nRachelside, ...",6.9289193560000015,NaN,f,NaN,0.548669,NaN,26.559749,0.0,...,<=50K,4.382943,52.0,0.0,Private,White,United-States,HS-grad,Married-civ-spouse,Husband
3851,Antonio Bailey,"29028 Bowman Parkways\r\nKerrview, FL 57851",0.368701087,Sales,f,6.0,10.093518,57.312500,79.734804,0.0,...,<=50K,96.128763,40.0,NaN,Self-emp-inc,White,Canada,10th,Widowed,Unmarried
2222,Antonio Bailey,"29028 Bowman Parkways\r\nKerrview, FL 57851",0.368701087,Sales,NaN,NaN,10.093518,57.312500,79.734804,NaN,...,<=50K,NaN,40.0,0.0,Self-emp-inc,White,Canada,10th,Widowed,Unmarried
2606,Antonio Rowan,"016 Jacob Tunnel Apt. 031\r\nAmandabury, CA 37683",??,Handlers-cleaners,f,9.0,0.208850,128.968750,15.056887,0.0,...,NaN,1.887960,45.0,0.0,Private,NaN,NaN,NaN,NaN,NaN
1987,Antonio Rowan,"016 Jacob Tunnel Apt. 031\r\nAmandabury, CA 37683",??,Handlers-cleaners,NaN,9.0,0.208850,128.968750,15.056887,0.0,...,<=50K,1.887960,45.0,0.0,Private,Black,United-States,HS-grad,Separated,Unmarried
3947,Ben Nelson,"45315 Obrien Inlet\r\nCynthiaport, OH 63603",22.14937084,Transport-moving,f,9.0,1.137467,92.804688,9.089983,0.0,...,>50K,0.511706,40.0,0.0,Private,White,United-States,HS-grad,Married-civ-spouse,Husband
956,Ben Nelson,"45315 Obrien Inlet\r\nCynthiaport, OH 63603",22.14937084,Transport-moving,f,9.0,NaN,NaN,NaN,0.0,...,>50K,0.511706,40.0,0.0,Private,NaN,NaN,NaN,NaN,NaN


Po vypísani duplicitných mien sme zistili, že zlúčením týchto duplikátov vieme vyplniť chýbajúce hodnoty. Zároveň hodnoty, ktoré majú oba duplikáty sú rovnaké. 

In [1271]:
dataO=fusionDuplicates(dataO)
dataO

,name,address,kurtosis_oxygen,occupation,pregnant,education-num,skewness_glucose,mean_glucose,std_oxygen,capital-gain,...,income,mean_oxygen,hours-per-week,capital-loss,workclass,ethn,state,edu,status,fam_memb
0,Daniel Love,"126 Ramirez Roads\r\nNew Jayville, IL 18861",10.58036423,Sales,f,9.0,3.158830,85.859375,14.121127,0.0,...,>50K,1.857860,40.0,1902.0,Private,White,United-States,HS-grad,Married-civ-spouse,Husband
1,Roy Paradis,"01168 Stark Pine Apt. 583\r\nDavidville, IN 32044",3.713364594,Craft-repair,f,9.0,1.234184,91.679688,36.946291,0.0,...,<=50K,10.001672,40.0,2002.0,Self-emp-not-inc,White,United-States,HS-grad,Married-civ-spouse,Husband
2,Charles Chen,"09482 Horne Lakes Apt. 373\r\nEast Brian, TN 2...",6.478064004,Handlers-cleaners,f,9.0,1.872355,89.960938,22.646980,0.0,...,<=50K,4.134615,48.0,0.0,Private,White,United-States,HS-grad,Married-civ-spouse,Husband
3,Chester Holmes,"720 Harris Turnpike Suite 732\r\nEast Sierra, ...",??,Sales,f,10.0,0.395583,NaN,14.001759,0.0,...,>50K,1.849498,40.0,0.0,Private,White,United-States,Some-college,Married-civ-spouse,Husband
4,James Ayala,Unit 3230 Box 9523\r\nDPO AA 55810,NaN,Transport-moving,f,9.0,0.493384,94.593750,29.361953,0.0,...,<=50K,5.739130,40.0,0.0,Private,Amer-Indian-Eskimo,United-States,HS-grad,Married-civ-spouse,Husband
5,Todd Flanagan,"2167 Wells Shoals Apt. 264\r\nEast Brianshire,...",6.384963147000001,?,f,9.0,-0.303185,125.742188,29.664529,0.0,...,<=50K,4.951505,15.0,0.0,?,White,United-States,HS-grad,Married-civ-spouse,Own-child
6,James Huffman,"535 Michael Squares\r\nNorth Sara, ID 61368",8.938092965,Sales,f,10.0,0.091356,145.835938,19.440728,0.0,...,<=50K,2.637124,50.0,0.0,Private,White,United-States,Some-college,Divorced,Not-in-family
7,Gary Prentice,"055 Nolan Club\r\nWest Bryanton, UT 15308",5.155939859,?,f,12.0,0.046797,138.179688,31.576347,0.0,...,<=50K,6.330268,1.0,0.0,?,White,United-States,Assoc-acdm,Divorced,Unmarried
8,Rodger Riis,7719 Frances Drives Apt. 525\r\nWest Brandonst...,2.436555082,Sales,f,10.0,-0.797075,129.976562,56.661158,0.0,...,<=50K,22.038462,35.0,0.0,Self-emp-inc,White,United-States,Some-college,Never-married,Own-child
9,Robert Cabiness,"70838 Robbins Camp Suite 853\r\nJacobville, SC...",14.38934822,Sales,f,9.0,1.035747,107.578125,11.797305,0.0,...,<=50K,1.002508,40.0,0.0,Private,White,United-States,HS-grad,Never-married,Own-child


In [1272]:
dataO.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933 entries, 0 to 3932
Data columns (total 25 columns):
name                3933 non-null object
address             3933 non-null object
kurtosis_oxygen     3677 non-null object
occupation          3933 non-null object
pregnant            3933 non-null object
education-num       3933 non-null float64
skewness_glucose    3933 non-null float64
mean_glucose        3539 non-null float64
std_oxygen          3933 non-null float64
capital-gain        3933 non-null float64
skewness_oxygen     3933 non-null float64
kurtosis_glucose    3933 non-null float64
fnlwgt              3933 non-null float64
class               3933 non-null float64
std_glucose         3932 non-null float64
income              3932 non-null object
mean_oxygen         3933 non-null float64
hours-per-week      3932 non-null float64
capital-loss        3933 non-null float64
workclass           3931 non-null object
ethn                3932 non-null object
state              

In [1273]:
dataO[dataO.isnull().any(axis=1)]

,name,address,kurtosis_oxygen,occupation,pregnant,education-num,skewness_glucose,mean_glucose,std_oxygen,capital-gain,...,income,mean_oxygen,hours-per-week,capital-loss,workclass,ethn,state,edu,status,fam_memb
3,Chester Holmes,"720 Harris Turnpike Suite 732\r\nEast Sierra, ...",??,Sales,f,10.0,0.395583,NaN,14.001759,0.0,...,>50K,1.849498,40.0,0.0,Private,White,United-States,Some-college,Married-civ-spouse,Husband
4,James Ayala,Unit 3230 Box 9523\r\nDPO AA 55810,NaN,Transport-moving,f,9.0,0.493384,94.593750,29.361953,0.0,...,<=50K,5.739130,40.0,0.0,Private,Amer-Indian-Eskimo,United-States,HS-grad,Married-civ-spouse,Husband
12,Thomas Rathburn,"708 Rodgers Streets Suite 909\r\nVanessamouth,...",8.620895914,Craft-repair,f,9.0,-0.685681,NaN,19.802528,0.0,...,<=50K,2.981605,40.0,0.0,Private,White,United-States,HS-grad,Married-civ-spouse,Husband
30,Patrick Cortez,"96216 Prince Prairie\r\nSouth Ryantown, IL 02827",7.1463000810000015,Exec-managerial,f,14.0,-0.036529,NaN,25.228435,0.0,...,<=50K,3.768395,45.0,0.0,Local-gov,White,United-States,Masters,Married-civ-spouse,Husband
56,Bobby Alicea,"787 Edwards Groves\r\nNew Melissaville, VT 70198",9.805805146,Exec-managerial,f,13.0,0.137797,NaN,18.641943,0.0,...,>50K,2.074415,40.0,0.0,Private,White,United-States,Bachelors,Married-civ-spouse,Husband
63,Kendrick Presha,9027 Odom Turnpike Suite 930\r\nLake Mariebury...,NaN,Other-service,f,7.0,2.432726,90.304688,12.879898,0.0,...,<=50K,2.207358,15.0,0.0,Private,White,United-States,11th,Never-married,Own-child
68,Fredrick Jack,"610 Griffith Causeway Apt. 479\r\nMatthewside,...",NaN,Machine-op-inspct,f,9.0,38.708198,7.796875,55.704857,0.0,...,<=50K,109.125418,20.0,0.0,Private,White,United-States,HS-grad,Never-married,Own-child
71,James Bonner,"09320 Smith Trace Apt. 263\r\nAllenfort, MI 15391",NaN,Machine-op-inspct,f,10.0,-0.551288,119.960938,12.605750,0.0,...,<=50K,1.769231,45.0,0.0,Private,White,United-States,Some-college,Married-civ-spouse,Husband
75,Mark Barrera,"18561 Scott Orchard Apt. 325\r\nDavismouth, MT...",9.233192137,Handlers-cleaners,f,9.0,0.146675,NaN,14.620565,0.0,...,<=50K,2.423913,20.0,0.0,Private,White,United-States,HS-grad,Divorced,Own-child
76,Thomas Bohler,"0168 Linda Pine Suite 684\r\nSouth Adamside, A...",??,Prof-specialty,FALSE,12.0,0.476484,NaN,14.197757,0.0,...,<=50K,1.751672,36.0,0.0,State-gov,Asian-Pac-Islander,United-States,Assoc-acdm,Divorced,Unmarried


In [1274]:
replaced = dataO.replace(to_replace=r'^[0-9,\-].*[0-9]$',value='number',regex=True)
replaced.select_dtypes(include='object')

,name,address,kurtosis_oxygen,occupation,pregnant,income,workclass,ethn,state,edu,status,fam_memb
0,Daniel Love,"126 Ramirez Roads\r\nNew Jayville, IL 18861",number,Sales,f,>50K,Private,White,United-States,HS-grad,Married-civ-spouse,Husband
1,Roy Paradis,"01168 Stark Pine Apt. 583\r\nDavidville, IN 32044",number,Craft-repair,f,<=50K,Self-emp-not-inc,White,United-States,HS-grad,Married-civ-spouse,Husband
2,Charles Chen,"09482 Horne Lakes Apt. 373\r\nEast Brian, TN 2...",number,Handlers-cleaners,f,<=50K,Private,White,United-States,HS-grad,Married-civ-spouse,Husband
3,Chester Holmes,"720 Harris Turnpike Suite 732\r\nEast Sierra, ...",??,Sales,f,>50K,Private,White,United-States,Some-college,Married-civ-spouse,Husband
4,James Ayala,Unit 3230 Box 9523\r\nDPO AA 55810,NaN,Transport-moving,f,<=50K,Private,Amer-Indian-Eskimo,United-States,HS-grad,Married-civ-spouse,Husband
5,Todd Flanagan,"2167 Wells Shoals Apt. 264\r\nEast Brianshire,...",number,?,f,<=50K,?,White,United-States,HS-grad,Married-civ-spouse,Own-child
6,James Huffman,"535 Michael Squares\r\nNorth Sara, ID 61368",number,Sales,f,<=50K,Private,White,United-States,Some-college,Divorced,Not-in-family
7,Gary Prentice,"055 Nolan Club\r\nWest Bryanton, UT 15308",number,?,f,<=50K,?,White,United-States,Assoc-acdm,Divorced,Unmarried
8,Rodger Riis,7719 Frances Drives Apt. 525\r\nWest Brandonst...,number,Sales,f,<=50K,Self-emp-inc,White,United-States,Some-college,Never-married,Own-child
9,Robert Cabiness,"70838 Robbins Camp Suite 853\r\nJacobville, SC...",number,Sales,f,<=50K,Private,White,United-States,HS-grad,Never-married,Own-child


In [1275]:
replaced['kurtosis_oxygen'].unique()

array(['number', '??', nan], dtype=object)

In [1276]:
replaced['occupation'].unique()

array([' Sales', ' Craft-repair', ' Handlers-cleaners',
       ' Transport-moving', ' ?', ' Adm-clerical', ' Exec-managerial',
       ' Farming-fishing', ' Other-service', ' Prof-specialty',
       ' Tech-support', ' Machine-op-inspct', ' Exec_managerial',
       ' Other_service', ' Prof_specialty', ' Protective-serv',
       ' Craft_repair', ' Protective_serv', ' Adm_clerical',
       ' Machine_op_inspct', ' Farming_fishing', ' Priv-house-serv',
       ' Transport_moving', ' Handlers_cleaners', ' Priv_house_serv',
       ' Tech_support', ' Armed-Forces'], dtype=object)

In [1277]:
replaced['pregnant'].unique()

array(['f', 'F', 'FALSE', 't', 'TRUE', 'T'], dtype=object)

In [1278]:
replaced['education-num'].unique()

array([ 9., 10., 12., 13.,  7., 11., 14., 16.,  8.,  6.,  3., 15.,  4.,
        5.,  2.,  1.])

In [1279]:
replaced['edu'].unique()

array(['HS-grad ', 'Some-college ', 'Assoc-acdm ', 'Bachelors ', '11th ',
       'Assoc-voc ', 'Masters ', 'Doctorate ', '12th ', '10th ',
       '5th-6th ', 'Prof-school ', '7th-8th ', '9th ', '1st-4th ',
       'Preschool ', nan], dtype=object)

In [1280]:
replaced['income'].unique()

array([' >50K', ' <=50K', nan], dtype=object)

In [1281]:
replaced['workclass'].unique()

array([' Private', ' Self-emp-not-inc', ' ?', ' Self-emp-inc',
       ' Federal-gov', ' State-gov', ' Local-gov', ' Without-pay', nan],
      dtype=object)

In [1282]:
replaced['ethn'].unique()

array(['White', 'Amer-Indian-Eskimo', 'Black', 'Asian-Pac-Islander',
       'Other', nan], dtype=object)

In [1283]:
replaced['state'].unique()

array(['United-States', 'India', 'Thailand', 'Puerto-Rico', 'Philippines',
       'Cuba', '?', 'China', 'Columbia', 'Canada', 'Poland', 'Mexico',
       'Vietnam', 'El-Salvador', 'Jamaica', 'Japan', 'Yugoslavia',
       'Italy', 'Germany', 'Guatemala', 'Ecuador', 'Hungary', 'Ireland',
       'Taiwan', 'England', 'France', 'Peru', 'Portugal', 'South',
       'Trinadad&Tobago', 'Dominican-Republic', 'Iran', 'Cambodia',
       'Outlying-US(Guam-USVI-etc)', 'Haiti', 'Scotland', 'Nicaragua',
       'Hong', 'Holand-Netherlands', 'Greece', nan], dtype=object)

In [1284]:
replaced['status'].unique()

array([' Married-civ-spouse', ' Divorced', ' Never-married', ' Separated',
       ' Widowed', ' Married-spouse-absent', ' Married-AF-spouse', nan],
      dtype=object)

In [1285]:
replaced['fam_memb'].unique()

array(['Husband', 'Own-child', 'Not-in-family', 'Unmarried', 'Wife',
       'Other-relative', nan], dtype=object)

In [1286]:
correctValuesO(dataO)

In [1287]:
dataO['pregnant'].unique()

array(['f', 't'], dtype=object)

In [1288]:
dataP.head(26)

,name,address,age,sex,date_of_birth
0,Roscoe Bohannon,"7183 Osborne Ways Apt. 651\r\nEast Andrew, OH ...",36,Female,1983-01-16
1,Ernest Kline,"391 Ball Road Suite 961\r\nFlowersborough, IN ...",4800,Female,71-09-03
2,Harold Hendriks,"8702 Vincent Square\r\nNew Jerryfurt, CO 30614",6200,Male,1957-10-02
3,Randy Baptiste,"2751 Harris Crossroad\r\nWest Ashley, CA 30311",1700,Male,2002-02-27
4,Anthony Colucci,"904 Robert Cliffs Suite 186\r\nWest Kyle, CO 7...",35,Male,1984-03-23
5,Ronald Lange,"30973 Martinez Shores\r\nJameston, CA 70245",60,Female,1959-01-07
6,Boyd Eiselein,29941 Christopher Curve Apt. 682\r\nRaychester...,49,Male,1970-09-12
7,Raymond Smith,53487 Scott Extensions Apt. 824\r\nMccartytown...,43,Female,1976-04-18
8,Harold Miller,"8514 Elizabeth Crescent\r\nWest Joseland, GA 4...",40,Male,1978-10-31
9,Charles Czachorowski,"6798 Wagner Locks Suite 377\r\nLake Brenda, DC...",58,Male,61-01-05


In [1289]:
replaced = dataP.replace(to_replace = r'[0-9]', value = 'x', regex = True)

In [1290]:
replaced["date_of_birth"].unique()

array(['xxxx-xx-xx', 'xx-xx-xx', 'xx/xx/xxxx', 'xxxx-xx-xx xx:xx:xx',
       'xxxx-xx-xx xx xx xx', 'xxxx/xx/xx'], dtype=object)

xx/xx/xxxx --> dd/mm/yyyy

In [1291]:
date = dataP["date_of_birth"][dataP["date_of_birth"].str.match(r'[0-9][0-9]/[0-9][0-9]/[0-9][0-9][0-9][0-9]')].sort_values(ascending=False).str.split("/", 3, expand = True)

In [1292]:
date.max()

0      31.0
1      12.0
2    2005.0
dtype: float64

xxxx/xx/xx --> yyyy/mm/dd

In [1293]:
date = dataP["date_of_birth"][dataP["date_of_birth"].str.match(r'[0-9][0-9][0-9][0-9]/[0-9][0-9]/[0-9][0-9]')].sort_values(ascending=False).str.split("/", 3, expand = True)

In [1294]:
date.max()

0    2020.0
1      12.0
2      31.0
dtype: float64

xxxx-xx-xx --> yyyy-mm-dd

In [1295]:
date = dataP["date_of_birth"][dataP["date_of_birth"].str.match(r'[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]')].sort_values(ascending=False).str.split("-", 3, expand = True)

In [1296]:
date.max()

0           2016
1             12
2    31 00:00:00
dtype: object

xx-xx-xx --> yy-mm-dd

In [1297]:
date = dataP["date_of_birth"][dataP["date_of_birth"].str.match(r'[0-9][0-9]-[0-9][0-9]-[0-9][0-9]')].sort_values(ascending=False).str.split("-", 3, expand = True)

In [1298]:
date.max()

0    96.0
1    12.0
2    31.0
dtype: float64

In [1299]:
date[0].unique()

array(['96', '93', '91', '90', '89', '88', '87', '86', '85', '84', '83',
       '82', '81', '80', '79', '78', '77', '76', '75', '74', '73', '72',
       '71', '70', '69', '68', '67', '66', '65', '64', '63', '62', '61',
       '60', '59', '58', '57', '56', '55', '54', '53', '52', '51', '50',
       '49', '48', '47', '42', '39', '29', '03', '00'], dtype=object)

In [1300]:
correctValuesP(dataP)

0    0 1983
1      0 01
2      0 16
Name: 0, dtype: object
0    1 71
1    1 09
2    1 03
Name: 1, dtype: object
0    2 1957
1      2 10
2      2 02
Name: 2, dtype: object
0    3 2002
1      3 02
2      3 27
Name: 3, dtype: object
0    4 1984
1      4 03
2      4 23
Name: 4, dtype: object
0    5 1959
1      5 01
2      5 07
Name: 5, dtype: object
0    6 1970
1      6 09
2      6 12
Name: 6, dtype: object
0    7 1976
1      7 04
2      7 18
Name: 7, dtype: object
0    8 1978
1      8 10
2      8 31
Name: 8, dtype: object
0    9 61
1    9 01
2    9 05
Name: 9, dtype: object


C:\Users\Andrej\Anaconda3\envs\iau\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


0    10 1963
1      10 02
2      10 19
Name: 10, dtype: object
0    11 1969
1      11 09
2      11 24
Name: 11, dtype: object
0    12 1977
1      12 05
2      12 23
Name: 12, dtype: object
0    13 1976
1      13 08
2      13 20
Name: 13, dtype: object
0    14 1952
1      14 08
2      14 28
Name: 14, dtype: object
0    15 1967
1      15 09
2      15 13
Name: 15, dtype: object
0    16 1951
1      16 05
2      16 01
Name: 16, dtype: object
0    17 1953
1      17 12
2      17 07
Name: 17, dtype: object
0    18 1942
1      18 12
2      18 09
Name: 18, dtype: object
0    19 1961
1      19 10
2      19 20
Name: 19, dtype: object
0    20 1971
1      20 02
2      20 17
Name: 20, dtype: object
0    21 1953
1      21 03
2      21 02
Name: 21, dtype: object
0    22 1959
1      22 12
2      22 23
Name: 22, dtype: object
0      23 24
1      23 09
2    23 1992
Name: 23, dtype: object
0           24 1960
1             24 10
2    24 11 00:00:00
Name: 24, dtype: object
0           25 1955
1             

Name: 138, dtype: object
0    139 1970
1      139 02
2      139 22
Name: 139, dtype: object
0    140 1967
1      140 01
2      140 26
Name: 140, dtype: object
0    141 73
1    141 12
2    141 22
Name: 141, dtype: object
0    142 1975
1      142 12
2      142 30
Name: 142, dtype: object
0    143 1973
1      143 11
2      143 27
Name: 143, dtype: object
0           144 1957
1             144 01
2    144 23 00 00 00
Name: 144, dtype: object
0    145 1966
1      145 07
2      145 25
Name: 145, dtype: object
0    146 1970
1      146 09
2      146 10
Name: 146, dtype: object
0    147 1968
1      147 07
2      147 27
Name: 147, dtype: object
0      148 06
1      148 11
2    148 1980
Name: 148, dtype: object
0    149 1992
1      149 05
2      149 02
Name: 149, dtype: object
0    150 1957
1      150 10
2      150 17
Name: 150, dtype: object
0    151 1953
1      151 11
2      151 07
Name: 151, dtype: object
0    152 1979
1      152 11
2      152 09
Name: 152, dtype: object
0    153 1958
1      1

0           266 1966
1             266 12
2    266 03 00 00 00
Name: 266, dtype: object
0    267 80
1    267 07
2    267 18
Name: 267, dtype: object
0    268 1980
1      268 08
2      268 22
Name: 268, dtype: object
0    269 70
1    269 02
2    269 10
Name: 269, dtype: object
0    270 1979
1      270 12
2      270 26
Name: 270, dtype: object
0    271 1980
1      271 08
2      271 07
Name: 271, dtype: object
0    272 1948
1      272 07
2      272 09
Name: 272, dtype: object
0           273 1974
1             273 02
2    273 28 00 00 00
Name: 273, dtype: object
0    274 1957
1      274 08
2      274 13
Name: 274, dtype: object
0    275 1963
1      275 09
2      275 18
Name: 275, dtype: object
0    276 1989
1      276 09
2      276 19
Name: 276, dtype: object
0    277 1972
1      277 04
2      277 26
Name: 277, dtype: object
0      278 18
1      278 12
2    278 1962
Name: 278, dtype: object
0    279 1961
1      279 09
2      279 10
Name: 279, dtype: object
0    280 1964
1      280 04
2   

...

0           3474 1956
1             3474 11
2    3474 07 00:00:00
Name: 3474, dtype: object
0           3475 1983
1             3475 12
2    3475 01 00:00:00
Name: 3475, dtype: object
0           3476 1968
1             3476 10
2    3476 13 00 00 00
Name: 3476, dtype: object
0    3477 1960
1      3477 05
2      3477 01
Name: 3477, dtype: object
0    3478 1958
1      3478 04
2      3478 16
Name: 3478, dtype: object
0    3479 1961
1      3479 02
2      3479 23
Name: 3479, dtype: object
0    3480 1970
1      3480 05
2      3480 12
Name: 3480, dtype: object
0           3481 1964
1             3481 04
2    3481 30 00:00:00
Name: 3481, dtype: object
0    3482 1946
1      3482 09
2      3482 24
Name: 3482, dtype: object
0      3483 09
1      3483 05
2    3483 1974
Name: 3483, dtype: object
0      3484 19
1      3484 05
2    3484 1983
Name: 3484, dtype: object
0           3485 1979
1             3485 02
2    3485 19 00:00:00
Name: 3485, dtype: object
0    3486 1965
1      3486 06
2      3486 1

0    3587 1954
1      3587 11
2      3587 02
Name: 3587, dtype: object
0    3588 1964
1      3588 06
2      3588 03
Name: 3588, dtype: object
0    3589 1976
1      3589 05
2      3589 31
Name: 3589, dtype: object
0    3590 1951
1      3590 05
2      3590 13
Name: 3590, dtype: object
0    3591 1962
1      3591 02
2      3591 20
Name: 3591, dtype: object
0    3592 1954
1      3592 12
2      3592 15
Name: 3592, dtype: object
0    3593 68
1    3593 09
2    3593 12
Name: 3593, dtype: object
0      3594 27
1      3594 01
2    3594 1979
Name: 3594, dtype: object
0    3595 1971
1      3595 07
2      3595 30
Name: 3595, dtype: object
0    3596 1963
1      3596 08
2      3596 04
Name: 3596, dtype: object
0    3597 1968
1      3597 04
2      3597 26
Name: 3597, dtype: object
0    3598 1968
1      3598 09
2      3598 12
Name: 3598, dtype: object
0    3599 1956
1      3599 07
2      3599 14
Name: 3599, dtype: object
0    3600 1973
1      3600 08
2      3600 20
Name: 3600, dtype: object
0    3601 19

Name: 3699, dtype: object
0           3700 1963
1             3700 05
2    3700 21 00:00:00
Name: 3700, dtype: object
0    3701 1962
1      3701 11
2      3701 24
Name: 3701, dtype: object
0    3702 1973
1      3702 07
2      3702 06
Name: 3702, dtype: object
0    3703 1962
1      3703 04
2      3703 08
Name: 3703, dtype: object
0    3704 1972
1      3704 09
2      3704 09
Name: 3704, dtype: object
0    3705 1978
1      3705 01
2      3705 12
Name: 3705, dtype: object
0           3706 1964
1             3706 09
2    3706 16 00:00:00
Name: 3706, dtype: object
0    3707 62
1    3707 07
2    3707 10
Name: 3707, dtype: object
0    3708 1971
1      3708 04
2      3708 15
Name: 3708, dtype: object
0      3709 11
1      3709 12
2    3709 1987
Name: 3709, dtype: object
0    3710 1972
1      3710 11
2      3710 28
Name: 3710, dtype: object
0    3711 1958
1      3711 05
2      3711 31
Name: 3711, dtype: object
0    3712 1954
1      3712 01
2      3712 12
Name: 3712, dtype: object
0    3713 1967


0    3815 1963
1      3815 03
2      3815 10
Name: 3815, dtype: object
0    3816 1974
1      3816 11
2      3816 02
Name: 3816, dtype: object
0    3817 1981
1      3817 12
2      3817 24
Name: 3817, dtype: object
0    3818 1971
1      3818 12
2      3818 16
Name: 3818, dtype: object
0    3819 1956
1      3819 04
2      3819 14
Name: 3819, dtype: object
0           3820 1969
1             3820 04
2    3820 01 00:00:00
Name: 3820, dtype: object
0    3821 1977
1      3821 08
2      3821 30
Name: 3821, dtype: object
0    3822 1974
1      3822 08
2      3822 20
Name: 3822, dtype: object
0    3823 1958
1      3823 12
2      3823 30
Name: 3823, dtype: object
0    3824 1971
1      3824 12
2      3824 04
Name: 3824, dtype: object
0    3825 1960
1      3825 06
2      3825 01
Name: 3825, dtype: object
0           3826 1957
1             3826 06
2    3826 10 00:00:00
Name: 3826, dtype: object
0    3827 1979
1      3827 04
2      3827 01
Name: 3827, dtype: object
0      3828 03
1      3828 04
2    

0    3927 1966
1      3927 09
2      3927 01
Name: 3927, dtype: object
0    3928 1968
1      3928 09
2      3928 22
Name: 3928, dtype: object
0    3929 1950
1      3929 11
2      3929 11
Name: 3929, dtype: object
0    3930 1988
1      3930 06
2      3930 19
Name: 3930, dtype: object
0    3931 1959
1      3931 10
2      3931 19
Name: 3931, dtype: object
0    3932 1997
1      3932 09
2      3932 25
Name: 3932, dtype: object


In [1301]:
dataP['date_of_birth']

0       1983-01-16
1       1971-09-03
2       1957-10-02
3       2002-02-27
4       1984-03-23
5       1959-01-07
6       1970-09-12
7       1976-04-18
8       1978-10-31
9       1961-01-05
10      1963-02-19
11      1969-09-24
12      1977-05-23
13      1976-08-20
14      1952-08-28
15      1967-09-13
16      1951-05-01
17      1953-12-07
18      1942-12-09
19      1961-10-20
20      1971-02-17
21      1953-03-02
22      1959-12-23
23      1992-09-24
24      1960-10-11
25      1955-04-30
26      1967-09-25
27      1978-07-21
28      1955-05-15
29      1956-03-16
           ...    
3903    1955-11-26
3904    1973-04-12
3905    1987-06-10
3906    1959-07-01
3907    1963-08-19
3908    1965-07-17
3909    1981-10-26
3910    1974-02-04
3911    1957-02-02
3912    1972-06-06
3913    1960-04-16
3914    1959-04-06
3915    1954-11-22
3916    1972-08-14
3917    1951-08-16
3918    1974-04-12
3919    1966-01-14
3920    1976-08-08
3921    1965-12-02
3922    1961-07-26
3923    1960-06-02
3924    1973

In [1302]:
replaced = dataP.replace(to_replace = r'[0-9]', value = 'x', regex = True)
replaced['date_of_birth'].unique()

array(['xxxx-xx-xx'], dtype=object)